In [1]:
# Clone the YOLOv7 repository
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7

# Install dependencies
!pip install -r requirements.txt

# Download pre-trained YOLOv7 weights
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt


c:\MSAAI\AAI-521\MSAAI-521-Final-Project\Sandbox\Gabriel\yolov7


Cloning into 'yolov7'...
c:\Users\gabri\anaconda3\envs\Torch_CV\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ------------------- -------------------- 7.1/14.6 MB 43.7 MB/s eta 0:00:01
   ---------------------------------------- 14.6/14.6 MB 46.0 MB/s eta 0:00:00
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
   ---------------------------------------- 0.0/525.0 kB ? eta -:--:--
   --------------------------------------- 525.0/525.0 kB 16.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 5.5/5.5 MB 84.5 MB/s eta 0:00:00
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 4.4/4.4 MB 133.

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import cv2
import torch
from yolov7.models.experimental import attempt_load
from yolov7.utils.general import non_max_suppression, scale_coords
from yolov7.utils.plots import plot_one_box
from yolov7.utils.datasets import letterbox

In [4]:
# Load YOLOv7 model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = attempt_load('yolov7.pt', map_location=device)
model.eval()

# Define the function to preprocess input frames
def preprocess_frame(frame, img_size=640):
    img = letterbox(frame, new_shape=img_size)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # Convert BGR to RGB and reshape
    img = torch.from_numpy(img).to(device).float() / 255.0  # Normalize
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
    return img

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


In [5]:
# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocess frame
    img = preprocess_frame(frame)
    
    # Perform inference
    pred = model(img)[0]
    pred = non_max_suppression(pred, 0.25, 0.45, classes=None, agnostic=False)
    
    # Draw bounding boxes
    for det in pred:
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], frame.shape).round()
            for *xyxy, conf, cls in det:
                label = f"Skin Cancer {conf:.2f}"
                plot_one_box(xyxy, frame, label=label, color=(255, 0, 0), line_thickness=2)
                
                # Save the detected region as an image
                detected_area = frame[int(xyxy[1]):int(xyxy[3]), int(xyxy[0]):int(xyxy[2])]
                cv2.imwrite("detected_area.jpg", detected_area)
                print("Detected area saved!")

    # Show the frame
    cv2.imshow("YOLOv7 Detection", frame)
    
    # Break on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ValueError: At least one stride in the given numpy array is negative, and tensors with negative strides are not currently supported. (You can probably work around this by making a copy of your array  with array.copy().) 